In [115]:
%load_ext sql
%sql postgresql://appdev@data/appdev

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


'Connected: appdev@appdev'

In [125]:
%%sql 
SELECT drivers.forename, drivers.surname, drivers.driverid, count(results.rank) as wins 
FROM drivers
JOIN results USING(driverid) 
WHERE results.rank = 1 
GROUP BY driverid
ORDER BY wins desc;

31 rows affected.


forename,surname,driverid,wins
Kimi,Räikkönen,8,41
Lewis,Hamilton,1,38
Sebastian,Vettel,20,31
Fernando,Alonso,4,22
Michael,Schumacher,30,21
Nico,Rosberg,3,20
Mark,Webber,17,18
Felipe,Massa,13,15
Daniel,Ricciardo,817,9
Jenson,Button,18,8


In [96]:
%%sql
SELECT drivers.driverref, constructors.name, count(*) from drivers
         JOIN results USING (driverid)
         JOIN constructors USING (constructorid)
         WHERE results.statusid = 1
         GROUP BY drivers.driverref, constructors.name
         ORDER BY count
         DESC
         LIMIT 10;

10 rows affected.


driverref,name,count
michael_schumacher,Ferrari,141
massa,Ferrari,112
rosberg,Mercedes,109
webber,Red Bull,100
vettel,Red Bull,97
raikkonen,Ferrari,92
button,McLaren,87
alonso,Ferrari,87
coulthard,McLaren,85
hamilton,Mercedes,85


In [112]:
%%sql
SELECT drivers.driverref, constructors.name, count(*) from drivers
          JOIN results USING (driverid)
          JOIN constructors USING (constructorid)
          WHERE results.statusid = 1
          GROUP BY drivers.driverref, constructors.name
          ORDER BY count
          DESC
          LIMIT 10;

10 rows affected.


driverref,name,count
michael_schumacher,Ferrari,141
massa,Ferrari,112
rosberg,Mercedes,109
webber,Red Bull,100
vettel,Red Bull,97
raikkonen,Ferrari,92
button,McLaren,87
alonso,Ferrari,87
coulthard,McLaren,85
hamilton,Mercedes,85


In [134]:
%%sql
select driverid, raceid, sum(milliseconds) as milli from pitstops
group by driverid, raceid
order by milli asc
limit 10;

10 rows affected.


driverid,raceid,milli
154,909,18928
17,877,19066
4,877,19368
18,877,19455
155,877,19529
13,877,19585
18,871,19668
822,955,19732
10,877,19884
3,955,20058


In [136]:
%%sql
create view pitstops_view as
select driverid, raceid, sum(milliseconds) as milli from pitstops
group by driverid, raceid
order by raceid asc;

Done.


[]

In [142]:
%%sql 
select drivers.driverref as driver, constructors.name as vehicle, count(constructorid) as car, 
sum(milli) as pitstop from drivers
join results using (driverid)
join pitstops_view using (driverid, raceid)
join constructors using (constructorid)
join status using (statusid) where status = 'Finished'
group by drivers.driverref, constructors.name
order by car desc
limit 10;

10 rows affected.


driver,vehicle,car,pitstop
rosberg,Mercedes,95,11501262
hamilton,Mercedes,85,12147994
button,McLaren,71,8515623
vettel,Red Bull,70,4927545
alonso,Ferrari,70,3691856
ricciardo,Red Bull,56,10948926
raikkonen,Ferrari,53,3608658
bottas,Williams,52,9184739
hulkenberg,Force India,49,9053044
webber,Red Bull,49,3017372


In [147]:
%%sql select drivers.driverref as driver, constructors.name as car, count(constructorid) as cardriven, 
sum(milli) as totalpitstop, round(sum(milli)/count(results.raceid), 0) as avgpitstop from drivers
join results using (driverid) 
join pitstops_view using (driverid, raceid)
join constructors using (constructorid)
join status USING (statusid) where status = 'Finished' 
group by drivers.driverref, constructors.name
order by avgpitstop ASC
limit 10;

10 rows affected.


driver,car,cardriven,totalpitstop,avgpitstop
ambrosio,Lotus F1,1,21962,21962
rosa,HRT,1,22272,22272
hulkenberg,Renault,5,186700,37340
glock,Marussia,2,82876,41438
bruno_senna,Williams,13,574413,44186
perez,Sauber,15,681053,45404
resta,Force India,34,1563408,45983
ricciardo,Toro Rosso,22,1015872,46176
raikkonen,Lotus F1,33,1532336,46434
maldonado,Williams,21,982946,46807
